In [1]:
import numpy as np
import numpy
import scipy
import qiskit
from qiskit import transpile
import scipy.linalg as sl
import numpy.linalg as nl

from scipy.linalg import cossin
from qiskit.quantum_info import Operator, Statevector
from qiskit import QuantumCircuit
from qiskit.circuit.library import UGate,UCPauliRotGate

import sys
sys.path.append('./src')

from lchs import *
from lcu import *

import utils_synth
import importlib
importlib.reload(utils_synth)
from utils_synth import *

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator


import oracle_synth
import importlib
importlib.reload(oracle_synth)
from oracle_synth import *

rng = numpy.random.default_rng(4290962056973923694427)
np.set_printoptions(precision=8, suppress=True)

In [ ]:
n = 3

psi = rng.random(2**n) - 0.5
psi = psi / numpy.linalg.norm(psi, ord=2)
print("state", psi)
##
state0 = numpy.zeros(2**n,dtype=int)
state0[0] = 1 #

my_ucr_circuit = QuantumCircuit(n)
stateprep_ucr(psi, my_ucr_circuit, debug=True)
my_ucr_circuit = my_ucr_circuit.reverse_bits() ## Hi, not gonna follow qiskit rule in my implementation

my_ucr_circuit_trans = transpile(my_ucr_circuit, basis_gates=['rz','ry','rx', 'cx'], optimization_level=0)
my_ucr_circuit_trans_opt = transpile(my_ucr_circuit, basis_gates=['rz','ry','rx', 'cx'], optimization_level=3)
print(my_ucr_circuit_trans.count_ops()['cx'], my_ucr_circuit_trans_opt.count_ops()['cx'])

In [ ]:
my_ucr_circuit.draw()

In [ ]:
my_ucr_circuit_trans_opt.draw()

In [13]:
from qiskit.synthesis.unitary.qsd import _get_ucry_cz

nqubits = 4
angles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8]

test_circ = _get_ucry_cz(nqubits, angles)
test_circ = test_circ.reverse_bits()
test_circ.draw()

┌──────────┐   ┌───────────┐      ┌──────────┐            ┌──────────┐
q_0: ┤ Ry(0.45) ├─■─┤ Ry(-0.05) ├─■──■─┤ Ry(-0.1) ├─■──■──■──■─┤ Ry(-0.2) ├
     └──────────┘ │ └───────────┘ │  │ └──────────┘ │  │  │  │ └──────────┘
q_1: ─────────────┼───────────────┼──┼──────────────■──┼──┼──┼─────────────
                  │               │  │                 │  │  │             
q_2: ─────────────┼───────────────■──┼─────────────────┼──■──┼─────────────
                  │                  │                 │     │             
q_3: ─────────────■──────────────────■─────────────────■─────■─────────────

In [14]:
import oracle_synth
import importlib
importlib.reload(oracle_synth)
from oracle_synth import *

## my implementation
circ2 = qiskit.QuantumCircuit(nqubits)
muxry_cz(circ2, angles, list(range(nqubits-1))[::-1], nqubits-1, angle_convert_flag=True)
circ2 = circ2.reverse_bits() ## not gonna follow qiskit rule in my implementation
circ2.draw()

┌──────────┐   ┌───────────┐      ┌──────────┐            ┌──────────┐
q_0: ┤ Ry(0.45) ├─■─┤ Ry(-0.05) ├─■──■─┤ Ry(-0.1) ├─■──■──■──■─┤ Ry(-0.2) ├
     └──────────┘ │ └───────────┘ │  │ └──────────┘ │  │  │  │ └──────────┘
q_1: ─────────────┼───────────────┼──┼──────────────■──┼──┼──┼─────────────
                  │               │  │                 │  │  │             
q_2: ─────────────┼───────────────■──┼─────────────────┼──■──┼─────────────
                  │                  │                 │     │             
q_3: ─────────────■──────────────────■─────────────────■─────■─────────────

In [4]:
## qiskit implementation
circ_qiskit = qiskit.QuantumCircuit(nqubits)
circ_qiskit.append(UCPauliRotGate(angles, 'Y'), list(range(nqubits)))
circ_qiskit.decompose(reps=2).draw()

┌──────────┐┌───┐┌───────────┐┌───┐┌───┐┌──────────┐┌───┐┌───┐┌───┐┌───┐»
q_0: ┤ Ry(0.45) ├┤ X ├┤ Ry(-0.05) ├┤ X ├┤ X ├┤ Ry(-0.1) ├┤ X ├┤ X ├┤ X ├┤ X ├»
     └──────────┘└─┬─┘└───────────┘└─┬─┘└─┬─┘└──────────┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘»
q_1: ──────────────■─────────────────┼────■────────────────┼────■────┼────■──»
                                     │                     │         │       »
q_2: ────────────────────────────────■─────────────────────┼─────────■───────»
                                                           │                 »
q_3: ──────────────────────────────────────────────────────■─────────────────»
                                                                             »
«     ┌──────────┐┌───┐
«q_0: ┤ Ry(-0.2) ├┤ X ├
«     └──────────┘└─┬─┘
«q_1: ──────────────┼──
«                   │  
«q_2: ──────────────┼──
«                   │  
«q_3: ──────────────■──
«

In [5]:

## my implementation
circ = qiskit.QuantumCircuit(nqubits)
multiplexer_pauli(circ, angles, list(range(nqubits-1)), nqubits-1, "Y")
circ = circ.reverse_bits() ## not gonna follow qiskit rule in my implementation
circ.draw()

┌──────────┐┌───┐┌───────────┐┌───┐┌───┐┌──────────┐┌───┐┌───┐┌───┐┌───┐»
q_0: ┤ Ry(0.45) ├┤ X ├┤ Ry(-0.05) ├┤ X ├┤ X ├┤ Ry(-0.1) ├┤ X ├┤ X ├┤ X ├┤ X ├»
     └──────────┘└─┬─┘└───────────┘└─┬─┘└─┬─┘└──────────┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘»
q_1: ──────────────■─────────────────┼────■────────────────┼────■────┼────■──»
                                     │                     │         │       »
q_2: ────────────────────────────────■─────────────────────┼─────────■───────»
                                                           │                 »
q_3: ──────────────────────────────────────────────────────■─────────────────»
                                                                             »
«     ┌──────────┐┌───┐
«q_0: ┤ Ry(-0.2) ├┤ X ├
«     └──────────┘└─┬─┘
«q_1: ──────────────┼──
«                   │  
«q_2: ──────────────┼──
«                   │  
«q_3: ──────────────■──
«